# ☸️ Helm Chart for AKS Deployment of RAG Web App

This notebook contains a full walkthrough to create a **Helm chart** for deploying your **Blazor/.NET RAG application** to **Azure Kubernetes Service (AKS)**.

We'll build:
- Helm directory structure
- `values.yaml` for config management
- Kubernetes manifests (Deployment, Service, Ingress)
- Deployment and upgrade commands for AKS

## 📁 Helm Chart Structure
```bash
rag-chart/
├── Chart.yaml
├── values.yaml
└── templates/
    ├── deployment.yaml
    ├── service.yaml
    └── ingress.yaml
```

In [ ]:
# Chart.yaml
apiVersion: v2
name: rag-web
description: A RAG .NET/Blazor app
version: 0.1.0
appVersion: "1.0.0"

In [ ]:
# values.yaml
replicaCount: 2
image:
  repository: youracr.azurecr.io/ragweb
  tag: latest
  pullPolicy: Always
service:
  type: LoadBalancer
  port: 80
ingress:
  enabled: true
  className: "nginx"
  hosts:
    - host: rag.example.com
      paths:
        - path: /
          pathType: ImplementationSpecific
env:
  - name: OPENAI_API_KEY
    value: "your-api-key"

In [ ]:
# templates/deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: {{ .Chart.Name }}
spec:
  replicas: {{ .Values.replicaCount }}
  selector:
    matchLabels:
      app: {{ .Chart.Name }}
  template:
    metadata:
      labels:
        app: {{ .Chart.Name }}
    spec:
      containers:
        - name: {{ .Chart.Name }}
          image: "{{ .Values.image.repository }}:{{ .Values.image.tag }}"
          imagePullPolicy: {{ .Values.image.pullPolicy }}
          env:
            {{- toYaml .Values.env | nindent 12 }}
          ports:
            - containerPort: 80

In [ ]:
# templates/service.yaml
apiVersion: v1
kind: Service
metadata:
  name: {{ .Chart.Name }}
spec:
  type: {{ .Values.service.type }}
  selector:
    app: {{ .Chart.Name }}
  ports:
    - protocol: TCP
      port: {{ .Values.service.port }}
      targetPort: 80

In [ ]:
# templates/ingress.yaml
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: {{ .Chart.Name }}
  annotations:
    kubernetes.io/ingress.class: {{ .Values.ingress.className }}
spec:
  rules:
    {{- range .Values.ingress.hosts }}
    - host: {{ .host }}
      http:
        paths:
          {{- range .paths }}
          - path: {{ .path }}
            pathType: {{ .pathType }}
            backend:
              service:
                name: {{ $.Chart.Name }}
                port:
                  number: 80
          {{- end }}
    {{- end }}

## 🚀 Deploy to AKS
```bash
az login
az aks get-credentials --resource-group <your-group> --name <your-cluster>

# Package and deploy
helm install ragweb ./rag-chart --values ./rag-chart/values.yaml

# Upgrade later
helm upgrade ragweb ./rag-chart --values ./rag-chart/values.yaml
```